It's actually just one now but ignore that lol

In [1]:
import pandas as pd
import numpy as np
import os
import json


In [2]:
bills = []
statesDict = {}
for f in os.walk(r"D:\Big Input Data Stuff\LegiScan\input"):
    if "bill" in f[0]:
        path = f[0].split("\\")
        if int(path[-2].split("_")[0].split("-")[0]) >= 2022:
            for file in f[-1]:
                if ".json" in file:
                    fileName = f[0] + "\\" + file
                    with open(fileName, "r") as b:
                        s = json.loads(b.read())
                        bills.append(s)
                        statesDict[s['bill']['state_id']] = s["bill"]["state"]

In [3]:
statuses = [
    "N\\A",
    "Intro",
    "Engrossed",
    "Enrolled",
    "Passed",
    "Vetoed",
    "Failed",
    "Override",
    "Chaptered",
    "Refer",
    "Report Pass",
    "Report DNP",
    "Draft",
]
bill_list = []
for bill in bills:
    bill_id = bill["bill"]["bill_id"]
    status = statuses[bill["bill"]["status"]]
    bill_name = bill["bill"]["title"]
    bill_number = bill["bill"]["bill_number"]
    bill_state = bill["bill"]["state"]
    bill_desc = bill["bill"]["description"]
    text_link = (
        bill["bill"]["texts"][-1]["url"] if len(bill["bill"]["texts"]) != 0 else ""
    )
    sponsors = []
    for person in bill["bill"]["sponsors"]:
        sponsors.append(person["people_id"])
    roll_call_ids = []
    for vote in bill["bill"]["votes"]:
        roll_call_ids.append(vote["roll_call_id"])
    progresses = []
    for progress in bill["bill"]["progress"]:
        progresses.append((progress["date"], progress["event"]))
    bill_list.append(
        (
            {
                "ID": bill_id,
                "Name": bill_name,
                "Number": bill_number,
                "State": bill_state,
                "Description": bill_desc,
                "Status": status,
                "Text": text_link,
                "Progress": f"{progresses}",
                "Sponsors": f"{sponsors}",
                "Votes": f"{roll_call_ids}",
            }
        )
    )



In [4]:
bill_df = pd.DataFrame.from_records(bill_list)
bill_df.set_index("ID", inplace=True)
bill_df


,Name,Number,State,Description,Status,Text,Progress,Sponsors,Votes
ID,,,,,,,,,
1648141,Repeal Ballot Measure 2 Voting Chgs,HB1,AK,An Act relating to elections.,Intro,https://legiscan.com/AK/text/HB1/id/2628184,"[('2023-01-19', 1), ('2023-01-19', 9)]","[18610, 21272, 22861, 21614]",[]
1647195,University: Textbooks/materials Cost,HB10,AK,An Act relating to costs of and charges for te...,Intro,https://legiscan.com/AK/text/HB10/id/2627240,"[('2023-01-19', 1), ('2023-01-19', 9)]","[23890, 24187]",[]
1730487,Paid Family Leave,HB100,AK,An Act relating to teacher and public employee...,Intro,https://legiscan.com/AK/text/HB100/id/2732779,"[('2023-03-08', 1), ('2023-03-08', 9)]","[24585, 22865]",[]
1730854,Campaign Funds For Dependent Care,HB101,AK,An Act relating to the use of campaign funds f...,Intro,https://legiscan.com/AK/text/HB101/id/2733051,"[('2023-03-08', 1), ('2023-03-08', 9)]","[24585, 18609, 24187]",[]
1730705,Address Confidentiality Program,HB102,AK,An Act relating to the duties of the Departmen...,Intro,https://legiscan.com/AK/text/HB102/id/2732941,"[('2023-03-08', 1), ('2023-03-08', 9)]",[24585],[]
...,...,...,...,...,...,...,...,...,...
1664730,Right of health care access-constitutional ame...,SJ0010,WY,A JOINT RESOLUTION proposing to amend the Wyom...,Failed,https://legiscan.com/WY/text/SJ0010/id/2646499,"[('2023-01-20', 1), ('2023-02-07', 6)]","[22676, 20193, 8711, 14226, 8675, 10602]",[1239782]
1664641,Convention of states.,SJ0011,WY,A JOINT RESOLUTION requesting Congress to call...,Engrossed,https://legiscan.com/WY/text/SJ0011/id/2646418,"[('2023-01-20', 1), ('2023-01-20', 9), ('2023-...","[24753, 23310, 24162, 23879, 18469, 23645, 234...","[1240828, 1243122, 1260993, 1263065]"
1666701,Constitutional amendment-property tax valuation.,SJ0012,WY,A JOINT RESOLUTION providing for the assessmen...,Failed,https://legiscan.com/WY/text/SJ0012/id/2648490,"[('2023-01-20', 1), ('2023-02-07', 6)]","[23643, 8679]",[1234868]


In [5]:
bill_df.to_pickle("./cleaned_input/bills_last_2_years.pkl.xz")